In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [2]:
base = pd.read_csv('test.csv')

In [3]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [4]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(60,)

In [5]:
y = y.values.reshape(-1,1)
y.shape

(60, 1)

In [6]:
y = onehot.fit_transform(y).toarray()
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0

In [7]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.3)

In [8]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((60, 100), (18, 100), (60, 3), (18, 3))

In [9]:
neuronios_entrada = x.shape[1]
neuronios_entrada

100

In [10]:
neuronios_oculta1 = int((neuronios_entrada + y.shape[1])/2)
neuronios_oculta1

51

In [32]:
neuronios_oculta2 = neuronios_oculta1
neuronios_oculta3 = neuronios_oculta1
neuronios_oculta4 = neuronios_oculta1
neuronios_oculta5 = neuronios_oculta1

In [12]:
neuronios_saida = y.shape[1]
neuronios_saida

3

In [13]:
# 100 -> 51 -> 51 -> 51 -> 3

In [33]:
w = {'oculta1': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta1])),
     'oculta2': tf.Variable(tf.random_normal([neuronios_oculta1, neuronios_oculta2])),
     'oculta3': tf.Variable(tf.random_normal([neuronios_oculta2, neuronios_oculta3])),
     'oculta4': tf.Variable(tf.random_normal([neuronios_oculta3, neuronios_oculta4])),
     'oculta5': tf.Variable(tf.random_normal([neuronios_oculta4, neuronios_oculta5])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta5, neuronios_saida]))}
w

{'oculta1': <tf.Variable 'Variable_8:0' shape=(100, 51) dtype=float32_ref>,
 'oculta2': <tf.Variable 'Variable_9:0' shape=(51, 51) dtype=float32_ref>,
 'oculta3': <tf.Variable 'Variable_10:0' shape=(51, 51) dtype=float32_ref>,
 'oculta4': <tf.Variable 'Variable_11:0' shape=(51, 51) dtype=float32_ref>,
 'oculta5': <tf.Variable 'Variable_12:0' shape=(51, 51) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_13:0' shape=(51, 3) dtype=float32_ref>}

In [34]:
b = {'oculta1': tf.Variable(tf.random_normal([neuronios_oculta1])),
     'oculta2': tf.Variable(tf.random_normal([neuronios_oculta2])),
     'oculta3': tf.Variable(tf.random_normal([neuronios_oculta3])),
     'oculta4': tf.Variable(tf.random_normal([neuronios_oculta4])),
     'oculta5': tf.Variable(tf.random_normal([neuronios_oculta5])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta1': <tf.Variable 'Variable_14:0' shape=(51,) dtype=float32_ref>,
 'oculta2': <tf.Variable 'Variable_15:0' shape=(51,) dtype=float32_ref>,
 'oculta3': <tf.Variable 'Variable_16:0' shape=(51,) dtype=float32_ref>,
 'oculta4': <tf.Variable 'Variable_17:0' shape=(51,) dtype=float32_ref>,
 'oculta5': <tf.Variable 'Variable_18:0' shape=(51,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_19:0' shape=(3,) dtype=float32_ref>}

In [35]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [36]:
def mlp(x, w, bias):
    camada_oculta1 = tf.nn.relu(tf.add(tf.matmul(x, w['oculta1']), b['oculta1']))
    camada_oculta2 = tf.nn.relu(tf.add(tf.matmul(camada_oculta1, w['oculta2']), b['oculta2']))
    camada_oculta3 = tf.nn.relu(tf.add(tf.matmul(camada_oculta2, w['oculta3']), b['oculta3']))
    camada_oculta4 = tf.nn.relu(tf.add(tf.matmul(camada_oculta2, w['oculta5']), b['oculta4']))
    camada_oculta5 = tf.nn.relu(tf.add(tf.matmul(camada_oculta2, w['oculta5']), b['oculta5']))
    camada_saida = tf.add(tf.matmul(camada_oculta3, w['saida']), b['saida'])
    return camada_saida

In [37]:
modelo = mlp(xph, w, b)

In [38]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(erro)

In [39]:
batch_size = 3
batch_total = int(len(x_treinamento) / batch_size)
batch_total

14

In [40]:
x_batches = np.array_split(x_treinamento, batch_total)
x_batches

[    Amostra 1  Amostra 2  Amostra 3  Amostra 4  Amostra 5  Amostra 6  \
 11        0.0      0.204      0.407      0.606      0.800      0.989   
 9         0.0      1.395      2.785      4.164      5.525      6.865   
 36        0.0      1.700      3.400      5.100      6.800      8.500   
 
     Amostra 7  Amostra 8  Amostra 9  Amostra 10  ...  Amostra 91  Amostra 92  \
 11      1.169      1.340      1.501       1.649  ...       2.277       2.239   
 9       8.177      9.455     10.696      11.894  ...     -11.894     -10.696   
 36     10.200     11.900     13.600      15.300  ...      17.000      18.700   
 
     Amostra 93  Amostra 94  Amostra 95  Amostra 96  Amostra 97  Amostra 98  \
 11       2.183       2.110       2.020       1.915       1.794       1.660   
 9       -9.455      -8.177      -6.865      -5.525      -4.164      -2.785   
 36      20.400      22.100      23.800      25.500      27.200      28.900   
 
     Amostra 99  Amostra 100  
 11       1.512        1.352  


In [41]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(3000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])

Época: 1 erro: 5794.033909388951
Época: 501 erro: 1.5972428083646912e-05
Época: 1001 erro: 4.427739673319918e-07
Época: 1501 erro: 1.1353262964673507e-08
Época: 2001 erro: 0.0
Época: 2501 erro: 0.0


In [23]:
w_final, b_final

({'oculta1': array([[-0.7120455 ,  0.4126431 , -1.377554  , ..., -0.05518564,
          -1.19602   , -1.8617177 ],
         [-0.46244308, -1.631183  , -0.55575544, ..., -0.97377676,
          -1.1841421 , -1.2703358 ],
         [-0.24545744, -0.7062301 ,  0.62997705, ...,  1.1979918 ,
          -0.8323742 , -0.21008891],
         ...,
         [-0.8765789 ,  0.5189366 , -1.103395  , ...,  1.2131436 ,
           1.1812949 ,  1.2904689 ],
         [ 0.75006646, -2.4998932 ,  1.0228422 , ..., -0.11909717,
          -0.14692935,  0.10767281],
         [-0.2103482 ,  1.5156806 , -2.2196414 , ..., -0.5118653 ,
           1.5998857 ,  0.00695339]], dtype=float32),
  'oculta2': array([[-0.90367234,  0.9757227 , -1.1466354 , ..., -1.3217337 ,
          -2.3740456 , -0.6189544 ],
         [ 1.7222602 ,  0.09177739, -0.65288633, ..., -1.7852818 ,
          -1.1059574 ,  0.2000637 ],
         [ 1.1922356 ,  2.55233   ,  0.11464572, ..., -0.3162944 ,
          -2.222422  ,  0.20370616],
         ..

In [24]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))

In [25]:
r1

array([[   672.7305  ,    490.53607 ,    167.99634 ],
       [  2041.7205  ,  -1831.8513  ,    582.8822  ],
       [  1027.9618  ,   -795.5342  ,    307.3267  ],
       [  -587.79865 ,   6342.9985  ,  -2120.1416  ],
       [ -2413.9587  ,   4233.0044  , -10224.803   ],
       [   644.9867  ,    280.6967  ,    212.9208  ],
       [  4579.1147  ,  -4406.0327  ,   1298.8787  ],
       [ -3288.9067  ,  -2057.4802  ,     51.451805],
       [  5085.9565  ,  -4921.2324  ,   1442.8978  ],
       [ -6054.096   ,  -3864.6306  ,     83.19866 ],
       [  1228.3967  ,   1264.2947  ,    760.48785 ],
       [  7523.9062  ,   4604.581   ,  -5033.8223  ],
       [ -5362.805   ,  -3412.8433  ,     75.26358 ],
       [   144.29019 ,   1403.6906  ,  -1529.9875  ],
       [  2951.972   ,   3111.883   ,   2087.271   ],
       [  4853.8745  ,   5126.465   ,   3524.9827  ],
       [  -570.8729  ,   -272.73914 ,    -12.88292 ],
       [  5091.622   ,   5378.293   ,   3704.7     ]], dtype=float32)

In [26]:
r2

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [2.5687317e-16, 1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [27]:
r3

array([0, 0, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0, 2, 1, 1, 1, 2, 1], dtype=int64)

In [28]:
y_teste

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [29]:
y_teste2 = np.argmax(y_teste, 1)
y_teste2

array([1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 2, 2, 2, 1, 1, 2, 1], dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto

0.7222222222222222